# Notebook I - Climate Regime
<hr>
This module performs climate data analysis and compiling general agro-climatic indicators. These general agro-climatic indicators summarize climatic profiles in the study area for each grid. The key input data for this module is the climatic data, and the geographical and terrain data.

Prepared by Geoinformatics Center, AIT
<hr>


---
### TO RUN THIS NOTEBOOK ON HPC ORION WITH GLOBAL DATA SIZE OF (2160,4320,366)

when starting your jupyter notebook session request:

- enter your fund number under "Account Name "
- Partition name = bigmem
- QOS = normal
- Number of hours = 
- Number of nodes = 
- Number of tasks = 
- Additional Slurm Parameters = --exclusive
- enter the path to your conda environment under "Additional Kernel Locations, colon separated"

---


Then, installing any additional python packages that required to run PyAEZ.
If working on your own PC/machine, these additional installation will vary depending on what is already installed in your Python library. 

In [ ]:
# 'Installing neccessary packages'
# !pip install gdal
# # !pip install pyaez==2.0.0
# additional packages used now: rioxarray, xarray, netcdcf4

Now, we will import the specific Python packages we need for PyAEZ.

In [1]:
'''import supporting libraries'''
import numpy as np
import matplotlib.pyplot as plt
import os
# try:
#     from osgeo import gdal
# except:
#     import gdal
import sys
from time import time as timer

# because I saved our data in nc files
# import xarray as xr

Setting the working directory -- where our PyAEZ project is located.

In [2]:
########################################################
### CHANGE THESE VALUES HERE FOR YOUR PERSONAL SETUP ###
########################################################

# branch version tag
revname='vectorize'

# 'Set the working directory'

# HPC Orion
# Replace with path to your PyAEZ folder under your username
work_dir = '/work/hpc/users/kerrie/UN_FAO/repos/PyAEZ/'

# Replace with whatever location you want to output data under your username
out_path = '/work/hpc/users/kerrie/UN_FAO/pyaez_results/global_1980/'+revname+'/' 

# these are the same for everyone on HPC Orion
# data_dir = '/work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/'
data_dir = '/work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/'
maskfile = '/work/hpc/datasets/un_fao/pyaez/static/netcdf/mask_2268708_5m.nc'
elevfile = '/work/hpc/datasets/un_fao/pyaez/static/netcdf/Elevation_2268708_5m.nc'


# # kerrie SSC desktop
# work_dir = 'K://projects/rgmg/climate/UN_FAO/repo_pyAEZ_kerrie/PyAEZ/'
# data_dir = 'C://Users/kerrie.WIN/Documents/data/pyAEZ_data_inputs_china_03272023/'
# out_path = work_dir+'kerrie/data_output/NB1/' 

# # kerrie laptop
# work_dir = 'C://Users/kerrie/Documents/01_LocalCode/repos/PyAEZ/' 
# data_dir = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/china/'
# out_path = work_dir+'kerrie/data_output/NB1/' 

########################################################
########################################################
########################################################

In [3]:
# # add pyaez model to system path
# sys.path.append(work_dir+'pyaez/') 

# Check whether the specified path exists or not
isExist = os.path.exists(out_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(out_path)
   print("The new directory is created!")

<hr>

## MODULE 1: CLIMATE REGIME
Now, we will start executing the routines in Module 1


First, we initiate Module 1 Class instance by invoking the following commands:

In [4]:
# # Import Module 1 and initate Class intance
# from pyaez import ClimateRegime
# clim_reg = ClimateRegime.ClimateRegime()

# # Importing UtilitiesCalc
# from pyaez import UtilitiesCalc
# obj_utilities = UtilitiesCalc.UtilitiesCalc()

import ClimateRegime_vec as ClimateRegime
clim_reg = ClimateRegime.ClimateRegime()

import UtilitiesCalc_vec as UtilitiesCalc
obj_utilities=UtilitiesCalc.UtilitiesCalc()

### Importing the climate dataset and the geographical data/rasters.

The package expects six climate variables, as daily or monthly observations, as Numpy arrays.
Arrays must be 3-dimensional, with the third axes containing the time dimension.
Unit of measures are expected as follows:
- Minimum temperature = Degree Celsius
- Maximum temperature = Degree Celsius
- Precipitation = Accumulated mm / day (or per month)
- Solar radiation = W/m$^2$
- Wind speed = Average m/s
- Relative humidity = Average fraction (0 to 1)

In addition to climate data, the system requires:
- A binary admin_mask, with 0 and 1 values. 0 pixels values will be not executed, while 1 pixels values will be executed
- An elevation layer
- Soil/terrain/special land cover classes
  

**All the datasets must have the same shape.**

In [24]:
# expect about 1 min run time
start=timer()
# Importing the climate data
max_temp = np.load(data_dir+'Tmax-2m/0.npy',mmap_mode="r")  # maximum temperature
min_temp = np.load(data_dir+'Tmin-2m/0.npy',mmap_mode="r")  # minimum temperature
precipitation = np.load(data_dir+'Precip/0.npy',mmap_mode="r")  # precipitation
rel_humidity = np.load(data_dir+'Rhum/0.npy')  # relative humidity
wind_speed = np.load(data_dir+'Wind-2m/0.npy') # wind speed measured at two meters
short_rad = np.load(data_dir+'Srad/0.npy')  # shortwave radiation

# Load the geographical data/rasters
mask=np.load(data_dir+'mask/0.npy')
elevation=np.load(data_dir+'Elevation/0.npy')

print(min_temp.shape)

task_time=timer()-start
task_time

(2160, 4320, 366)


14.694885492324829

In [6]:
# ###################################################################
# ### USE ABOVE INSTEAD AFTER REWRITING DATA TO NPY AND TIF ###
# ###################################################################


# start=timer()
# # Open the data files, this is quick
# # pyaez needs arrays shaped (ny,nx,nt) and
# # also, the spatial reference/geo info should at minimum be 
# # included in the mask input data 
# # (i.e. the input mask.nc should have spatial_ref or mask.tif should be geotif)
# max_temp = xr.open_dataset(data_dir+'Tmax-2m_daily_1980_5m.nc')['Tmax-2m'].transpose('lat','lon','time').data
# min_temp = xr.open_dataset(data_dir+'Tmin-2m_daily_1980_5m.nc')['Tmin-2m'].transpose('lat','lon','time').data
# precipitation = xr.open_dataset(data_dir+'Precip_daily_1980_5m.nc')['Precip'].transpose('lat','lon','time').data
# rel_humidity = xr.open_dataset(data_dir+'Rhum_daily_1980_5m.nc')['Rhum'].transpose('lat','lon','time').data
# wind_speed = xr.open_dataset(data_dir+'Wind-2m_daily_1980_5m.nc')['Wind-2m'].transpose('lat','lon','time').data
# short_rad = xr.open_dataset(data_dir+'Srad_daily_1980_5m.nc')['Srad'].transpose('lat','lon','time').data

# mask=xr.open_dataset(maskfile)['mask'].data
# elevation=xr.open_dataset(elevfile)['Elevation'].data

# print(max_temp.shape)

# task_time=timer()-start
# task_time



In [7]:
# Define the Area-Of-Interest's geographical extents

# This section contains parameters that can be modified by the user:
# - lat_min = minimum latitude of analysis
# - lat_max = maximum latitude of analysis
# - mask_value = the value in the admin_mask to exclude from the analysis (typically 0)
# - daily = whether climate input data are daily (True) or monthly (False)


# rebuilding the grid like this results in problems with precision and 
# slight shifting of all the grids
# why not just feed the entire array of latitudes in from the data to avoid this?
# lat_min = 18.00000000
# lat_max = 53.66666667
lats = xr.open_dataset(maskfile)['lat'].data
mask_path = maskfile # if using .nc, variable name inside file needs to match the string of letters before first "_" in file name
mask_value = 0  # pixel value in admin_mask to exclude from the analysis
daily = True #Type of climate data = True: daily, False: monthly


### Loading the imported data into the Object Class ('*clim_reg*' Class)

In [8]:
start=timer()
clim_reg.setStudyAreaMask(mask, mask_value)
task_time=timer()-start
task_time

6.127357482910156e-05

In [9]:
start=timer()
# clim_reg.setLocationTerrainData(lat_min, lat_max, elevation)
clim_reg.setLocationTerrainData(lats, elevation)
task_time=timer()-start
task_time

0.030428409576416016

In [10]:
# expect xx min run time
start=timer()
if daily:
    clim_reg.setDailyClimateData(
        min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)
else:
    clim_reg.setMonthlyClimateData(
        min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)
    # clim_reg.setMonthlyClimateData(
    #     min_temp.sel(time=slice(1,12)), max_temp.sel(time=slice(1,12)), \
    #     precipitation.sel(time=slice(1,12)), short_rad.sel(time=slice(1,12)), \
    #     wind_speed.sel(time=slice(1,12)), rel_humidity.sel(time=slice(1,12)))        
task_time=timer()-start
task_time  

assigning fake1
assigning fake2
assigning fake3
assigning fake4
assigning fake5
assigning fake6
assigning fake7
assigning fake8
assigning fake9
assigning fake10
assigning fake11
assigning fake12


30.165728330612183

In [11]:
'Releasing the memory of input climate data -- free up some RAM space'
del(min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)

NOTE: After loading the data into the *clim_reg* Class, all the parameters will be converted to DAILY DATA and calculated as other parameters. 
These new parameters are available and can be called into use as:
- *clim_reg.minT_daily* (minimum temperature)
- *clim_reg.maxT_daily* (maximum temperature)
- *clim_reg.meanT_daily* (mean temperature)
- *clim_reg.meanT_daily_sealevel* (mean temperature, corrected to sea-level)
- *clim_reg.totalPrec_daily* (total precipitation)
- *clim_reg.pet_daily* (reference evapotranspiration, ETo)
- *clim_reg.P_by_PET_daily* (ratio of precipitation over ETo)
  


In [13]:
clim_reg.__dict__.keys()


dict_keys(['im_mask', 'nodata_val', 'set_mask', 'elevation', 'im_height', 'im_width', 'latitude', 'meanT_daily', 'totalPrec_daily', 'maxT_daily', 'fake1_3D', 'fake2_3D', 'fake3_3D', 'fake4_3D', 'fake5_3D', 'fake6_3D', 'fake7_3D', 'fake8_3D', 'fake9_3D', 'fake10_3D', 'fake11_3D', 'fake12_3D'])

In [ ]:
# smoothed mean T
# Adding interpolation to the dataset
# 5th degree spline fit to smooth in time
print('compute meanT...')
meanT_daily = (min_temp+max_temp)/2.

print('create days...')
days = np.arange(1,367) # x values

print('create 3D mask')
# replace any nan with zero
mask3D = np.tile(mask[:,:,np.newaxis], (1,1,days.shape[0]))

print('replace nan with 0...')
data=np.where(mask3D==0,0,meanT_daily)
print(data.shape)

print('collapse to 2D...')
data2D=data.transpose(2,0,1).reshape(days.shape[0],-1) # every column is a set of y values
print(data.shape)

print('del data')
del data

print('quad_spl...')
# do the fitting
quad_spl=np.polynomial.polynomial.polyfit(days,data2D,deg=5)

print('interp_daily_temp...')
interp_daily_temp=np.polynomial.polynomial.polyval(days,quad_spl)

print('reshape interp_daily_temp...')
#reshape
interp_daily_temp=interp_daily_temp.reshape(mask3D.shape[0],mask3D.shape[1],-1)
print('finished')

compute meanT...
create days...
create 3D mask
replace nan with 0...
(2160, 4320, 366)
collapse to 2D...
del data
qual_spl...


In [21]:
clim_reg.get('meanT_daily')

AttributeError: 'ClimateRegime' object has no attribute 'get'

In [20]:
sys.getsizeof(clim_reg)/1E9

TypeError: 'ClimateRegime' object is not subscriptable

In [23]:
[(x, sys.getsizeof(x)/1E9) for x in dir(clim_reg)]

[('AEZClassification', 6.6e-08),
 ('AirFrostIndexandPermafrostEvaluation', 8.5e-08),
 ('TZoneFallowRequirement', 7.1e-08),
 ('__class__', 5.8e-08),
 ('__delattr__', 6e-08),
 ('__dict__', 5.7e-08),
 ('__dir__', 5.6e-08),
 ('__doc__', 5.6e-08),
 ('__eq__', 5.5e-08),
 ('__format__', 5.9e-08),
 ('__ge__', 5.5e-08),
 ('__getattribute__', 6.5e-08),
 ('__getstate__', 6.1e-08),
 ('__gt__', 5.5e-08),
 ('__hash__', 5.7e-08),
 ('__init__', 5.7e-08),
 ('__init_subclass__', 6.6e-08),
 ('__le__', 5.5e-08),
 ('__lt__', 5.5e-08),
 ('__module__', 5.9e-08),
 ('__ne__', 5.5e-08),
 ('__new__', 5.6e-08),
 ('__reduce__', 5.9e-08),
 ('__reduce_ex__', 6.2e-08),
 ('__repr__', 5.7e-08),
 ('__setattr__', 6e-08),
 ('__sizeof__', 5.9e-08),
 ('__str__', 5.6e-08),
 ('__subclasshook__', 6.5e-08),
 ('__weakref__', 6e-08),
 ('elevation', 5.8e-08),
 ('fake10_3D', 5.8e-08),
 ('fake11_3D', 5.8e-08),
 ('fake12_3D', 5.8e-08),
 ('fake1_3D', 5.7e-08),
 ('fake2_3D', 5.7e-08),
 ('fake3_3D', 5.7e-08),
 ('fake4_3D', 5.7e-08),
 ('

In [12]:
import sys

# These are the usual ipython objects, including this one 
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))/1E9) \
        for x in dir() \
        if not x.startswith('_') \
        and x not in sys.modules \
        and x not in ipython_vars], 
       key=lambda x: x[1], reverse=True)

[('elevation', 0.037324928),
 ('mask', 0.037324928),
 ('open', 1.52e-07),
 ('elevfile', 1.18e-07),
 ('out_path', 1.15e-07),
 ('mask_path', 1.13e-07),
 ('maskfile', 1.13e-07),
 ('lats', 1.12e-07),
 ('data_dir', 1.03e-07),
 ('work_dir', 9.1e-08),
 ('ClimateRegime', 7.2e-08),
 ('UtilitiesCalc', 7.2e-08),
 ('np', 7.2e-08),
 ('plt', 7.2e-08),
 ('timer', 7.2e-08),
 ('xr', 7.2e-08),
 ('revname', 5.8e-08),
 ('clim_reg', 5.6e-08),
 ('obj_utilities', 5.6e-08),
 ('daily', 2.8e-08),
 ('isExist', 2.8e-08),
 ('mask_value', 2.8e-08),
 ('start', 2.4e-08),
 ('task_time', 2.4e-08)]

#### Thermal Climate
The Thermal Climate function calculates and classifies latitudinal thermal climate, which will be used later in Module 2 for the assessment of potential crops and land utilization types (LUT) presence in each grid cell.

In [ ]:
tclimate = clim_reg.getThermalClimate()


In [ ]:

'''save and visualize result'''
fig = plt.figure()
plt.imshow(tclimate, cmap=plt.get_cmap('gist_ncar_r', 12),vmin=-0.3,vmax=12.5,interpolation='none')
plt.title('Thermal Climate')
plt.colorbar()
plt.savefig(out_path+"ThermalClimate.png",bbox_inches ="tight",dpi=300) #Save as PNG image
plt.show()

obj_utilities.saveRaster(mask_path, out_path+'ThermalClimate_'+revname+'.tif',tclimate) #Save as GeoTIFF raster

#### Thermal Zone
The thermal zone is classified based on actual temperature which reflects on the temperature regimes of major thermal climates

In [ ]:
tzone = clim_reg.getThermalZone()

In [ ]:

'''save and visualize result'''
fig = plt.figure()
plt.imshow(tzone, cmap=plt.get_cmap('gist_ncar_r', 12),vmin=-0.3,vmax=12.5,interpolation='none')
plt.title('Thermal Zones')
plt.colorbar()
plt.savefig(out_path+"ThermalZone.png",bbox_inches ="tight",dpi=300) #Save as PNG image
plt.show()

obj_utilities.saveRaster(mask_path, out_path+'ThermalZone_'+revname+'.tif',tzone) #Save as GeoTIFF raster


#### Thermal Length of Growing Period (LGP)
The thermal length of growing period (LGPt) is defined as the number of days in a year during which the daily mean temperature (Ta) is conductive to crop growth and development. PyAEZ utilizes the AEZ three standard temperature thresholds for LGPt:
- Periods with Ta>0°C (LGPt0)
- Periods with Ta>5°C (LGPt5) – the period conductive to plant growth and development
- Periods, and Ta>10°C (LGPt10) – a proxy for the period of low risks for late and early frost occurrences and termed ‘frost-free period’

In [ ]:
# 3 separate calls means three times as much compute time
# can these be combined, one call outputs all 3
lgpt0 = clim_reg.getThermalLGP0()
lgpt5 = clim_reg.getThermalLGP5()
lgpt10 = clim_reg.getThermalLGP10()

In [ ]:
'''save and visualize result'''
#======================
plt.figure(1, figsize=(24, 8))
plt.subplot(1, 3, 1)
plt.imshow(lgpt0,vmin=0,vmax=366,interpolation='none')
plt.title('LGPt 0')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 2)
plt.imshow(lgpt5, vmin=0, vmax=366,interpolation='none')
plt.title('LGPt 5')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 3)
plt.imshow(lgpt10, vmin=0, vmax=366,interpolation='none')
plt.title('LGPt 10')
plt.colorbar(shrink=0.8)
#----------------------
plt.savefig(out_path+"ThermalLGPs_"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
#======================

obj_utilities.saveRaster(mask_path, out_path+'LGPt0_'+revname+'.tif', lgpt0)
obj_utilities.saveRaster(mask_path, out_path+'LGPt5_'+revname+'.tif', lgpt5)
obj_utilities.saveRaster(mask_path, out_path+'LGPt10_'+revname+'.tif', lgpt10)


#### Temperature Sum

In [ ]:
tsum0 = clim_reg.getTemperatureSum0()
tsum5 = clim_reg.getTemperatureSum5()
tsum10 = clim_reg.getTemperatureSum10()

In [ ]:
'''save and visualize result'''
#======================
plt.figure(1, figsize=(24, 8))
plt.subplot(1, 3, 1)
plt.imshow(tsum0, cmap='hot_r', vmin=0, vmax=11000,interpolation='none')
plt.title('T-sumation 0')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 2)
plt.imshow(tsum5, cmap='hot_r', vmin=0, vmax=11000,interpolation='none')
plt.title('T-sumation 5')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 3)
plt.imshow(tsum10, cmap='hot_r', vmin=0, vmax=11000,interpolation='none')
plt.title('T-sumation 10')
plt.colorbar(shrink=0.8)
#----------------------
plt.savefig(out_path+"Tsum_"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
#======================

obj_utilities.saveRaster(mask_path, out_path+'tsum0_'+revname+'.tif', tsum0)
obj_utilities.saveRaster(mask_path, out_path+'tsum5_'+revname+'.tif', tsum5)
obj_utilities.saveRaster(mask_path, out_path+'tsum10_'+revname+'.tif', tsum10)


#### Temperature Profile

In [ ]:
tprofile = clim_reg.getTemperatureProfile()

In [ ]:
tile_list = ['A1','A2','A3','A4','A5','A6','A7','A8','A9', \
            'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9']

'''save and visualize result'''

fig = plt.figure(figsize=(12, 20))
for i,profile in enumerate(tprofile):
    plt.subplot(6, 3, i+1)
    plt.imshow(profile,interpolation='none')
    plt.title(tile_list[i])
    plt.colorbar(shrink=0.9)
plt.tight_layout()

plt.savefig(out_path+"Tprofiles_"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()

# for i1 in range(ntiles+1):
for tile,profile in zip(tile_list,tprofile):
    obj_utilities.saveRaster(
        mask_path, out_path+'TProfile_' + tile + '_'+revname+'.tif', profile)
    

#### Length of Growing Periods (LGPs)

In [ ]:
lgp = clim_reg.getLGP( Sa = 100 )
lgp_class = clim_reg.getLGPClassified(lgp)
lgp_equv = clim_reg.getLGPEquivalent()

In [ ]:
'''save and visualize result'''

plt.imshow(lgp, cmap='viridis', vmin=0, vmax=366,interpolation='none')
plt.title('LPG [days]')
plt.colorbar()
plt.savefig(out_path+"LGP_"+revname+".png", bbox_inches="tight", dpi=300)
plt.show()


plt.imshow(lgp_equv, cmap='viridis', vmin=0, vmax=366,interpolation='none')
plt.title('LPG Equivalent [days]')
plt.colorbar()
plt.savefig(out_path+"LGP_Equv"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()

obj_utilities.saveRaster(mask_path, out_path+'LGP_'+revname+'.tif', lgp)

obj_utilities.saveRaster(mask_path, out_path+'LGPEquivalent_'+revname+'.tif', lgp_equv)

#### Multi Cropping Zone
Multiple cropping zones classification is an additional agro-climatic indicator, which relates to the possibility of cultivating multiple sequential crops under rain-fed and irrigated conditions.


In [ ]:
multi_crop = clim_reg.getMultiCroppingZones(tclimate, lgp, lgpt5, lgpt10, tsum0, tsum10)
multi_crop_rainfed = multi_crop[0]  # for rainfed conditions
multi_crop_irr = multi_crop[1]  # for irrigated conditions

In [ ]:
'''save and visualize result'''

plt.imshow(multi_crop_irr, cmap=plt.get_cmap('gist_ncar_r', 9), vmin=-0.2, vmax=8.4,interpolation='none')
plt.title('Multi Cropping Zone - IRR')
plt.colorbar()
plt.savefig(out_path+"multicrop_irr_"+revname+".png", bbox_inches="tight", dpi=300)
plt.show()
obj_utilities.saveRaster(
    mask_path, out_path+'multicrop_irr_'+revname+'.tif', multi_crop_irr)


plt.imshow(multi_crop_rainfed,cmap=plt.get_cmap('gist_ncar_r', 9), vmin=-0.2, vmax=8.4,interpolation='none')
plt.title('Multi Cropping Zone - RAINFED')
plt.colorbar()
plt.savefig(out_path+"multicrop_rain_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()
obj_utilities.saveRaster(
    mask_path, out_path+'multicrop_rain_'+revname+'.tif', multi_crop_rainfed)


### Air Frost Index and Permafrost Evaluation
Occurrence of continuous or discontinuous permafrost conditions are used in the suitability assessment. Permafrost areas are characterized by sub-soil at or below the freezing point for two or more years. In this section, PyAEZ utilizes the air frost index (FI) which is used to characterize climate-derived permafrost condition into 4 classes: 
1) Continuous permafrost
2) Discontinuous permafrost 
3) Sporadic permafrost
4) No permafrost

In [ ]:
permafrost_eval = clim_reg.AirFrostIndexandPermafrostEvaluation()
frost_index = permafrost_eval[0]
permafrost = permafrost_eval[1]

In [ ]:
'''save and visualize result'''

plt.imshow(frost_index, cmap=plt.get_cmap(
    'tab20b', 11), vmin=-0.05, vmax=1.05,interpolation='none')
plt.title('Frost Index')
plt.colorbar()
plt.savefig(out_path+"frost_index_"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
obj_utilities.saveRaster(
    mask_path, out_path+'frost_index_'+revname+'.tif', frost_index)



plt.imshow(permafrost, cmap=plt.get_cmap(
    'tab20b', 5), vmin=-0.5, vmax=4.3,interpolation='none')
plt.title('Permafrost Evaluation')
plt.colorbar()
plt.savefig(out_path+"permafrost_"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
obj_utilities.saveRaster(
    mask_path, out_path+'permafrost_'+revname+'.tif', permafrost)


### Fallow period requirement
Fallow is an agricultural technique that consists of not sowing the arable land during one or more growing seasons. In AEZ framework, the fallow factors have been established by main crop groups and environmental conditions. The crop groups include cereals, legumes, roots and tubers, and a miscellaneous group consisting of long-term annuals/perennials. The fallow factors are expressed as percentage of time during the fallow-cropping cycle the land must be under fallow. PyAEZ determines the fallow requirements using Thermal Zones.


In [ ]:
tzone_fallow = clim_reg.TZoneFallowRequirement(tzone)


In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(tzone_fallow, cmap=plt.get_cmap('tab10', 7), vmin=-0.5, vmax=6.3,interpolation='none')
plt.title('Fallow Requirement')
plt.colorbar()
plt.savefig(out_path+"fallow_"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
obj_utilities.saveRaster(
    mask_path, out_path+'fallow_'+revname+'.tif', tzone_fallow)


### Agro-ecological zones classification
The agro-ecological zones (AEZ) methodology provides a framework for establishing a spatial inventory of land resources compiled from global/national environmental data sets and assembled to quantify multiple spatial characteristics required for the assessments of land productivity under location-specific agro-ecological conditions.


In [ ]:

aez = clim_reg.AEZClassification(
    tclimate, lgp, lgp_equv, lgpt5, soil_terrain_lulc, permafrost)

# now visualizing result
fig = plt.figure()
plt.imshow(aez, cmap=plt.get_cmap('rainbow', 59), vmin=0, vmax=59,interpolation='none')
plt.title('Agro-ecological Zonation')
plt.colorbar()
plt.savefig(out_path+"aez_"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
obj_utilities.saveRaster(
    mask_path, out_path+'aez_'+revname+'.tif', aez)


<hr>

### END OF MODULE 1: CLIMATE REGIME

<hr>